In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import gmaps.datasets
import requests
import matplotlib.pyplot as plt
# Import the API key.
from config import g_key
gmaps.configure(api_key=g_key)

In [2]:
# Store the CSV you saved created in part two into a DataFrame.
travel_df = pd.read_csv("WeatherPy_vacation.csv")
travel_df.head(50)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Chaman,PK,75.09,clear sky,30.92,66.45,Niamatt parda Center
1,6,Moroni,KM,76.24,clear sky,-11.70,43.26,Hotel les Arcades
2,7,Miri,MY,78.01,scattered clouds,4.41,114.01,"Mega Hotel, Miri"
3,10,Tirumullaivasal,IN,83.34,light rain,11.23,79.83,"MATHAMPATTINAM, MODUNUTTI GROUP"
4,14,Tezu,IN,75.06,moderate rain,27.92,96.17,Circuit House
5,16,Kenora,CA,75.20,shower rain,49.82,-94.43,Maynard Lake Lodge
6,18,Atuona,PF,80.60,clear sky,-9.80,-139.03,Villa Enata
7,19,Alofi,NU,75.20,overcast clouds,-19.06,-169.92,Taloa Heights
8,23,Sao Joao Da Barra,BR,75.20,overcast clouds,-21.64,-41.05,Pousada Porto De Canoas
9,25,Jamestown,US,75.00,clear sky,42.10,-79.24,Hampton Inn & Suites Jamestown


In [3]:
# Extract 4 cities:
city_A = travel_df.loc[travel_df.City == "San Patricio"].reset_index()
city_B = travel_df.loc[travel_df.City == "Coahuayana"].reset_index()
city_C = travel_df.loc[travel_df.City == "Lazaro Cardenas"].reset_index()
city_D = travel_df.loc[travel_df.City == "Acapulco"].reset_index()

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get location
locA = (city_A.loc[0,'Lat'], city_A.loc[0,'Lng'])
locB = (city_B.loc[0,'Lat'], city_B.loc[0,'Lng'])
locC = (city_C.loc[0,'Lat'], city_C.loc[0,'Lng'])
locD = (city_D.loc[0,'Lat'], city_D.loc[0,'Lng'])

In [6]:
# gmap travel:
fig = gmaps.figure()
drive = gmaps.directions_layer(locA, locD, waypoints=[locB, locC], travel_mode='DRIVING')
fig.add_layer(drive)
# Save the figure.
plt.savefig("WeatherPy_travel_map.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [7]:
# Combine all 4 cities into 1 df:
combined_df = city_A.append(city_B).append(city_C).append(city_D)
combined_df

,index,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,11,33,San Patricio,MX,89.60,scattered clouds,19.22,-104.70,Hotel Cabo Blanco
0,48,166,Coahuayana,MX,82.40,broken clouds,18.73,-103.68,Hotel Los Arcos
0,46,160,Lazaro Cardenas,MX,86.23,light rain,17.96,-102.20,Hotel Quinta Antigua
0,28,88,Acapulco,MX,87.80,broken clouds,16.86,-99.89,HS Hotsson Smart Acapulco


In [28]:
# Create pop-ups.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
travel_info = [info_box_template.format(**row) for index, row in travel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = travel_df[["Lat", "Lng"]]
max_temp = travel_df["Max Temp"]
fig = gmaps.figure(center=(19.0, -95.0), zoom_level=4.0)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Save the figure.
plt.savefig("WeatherPy_travel_map_markers.png")

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>